In [ ]:
import pandas as pd
import requests 
import json
from datetime import datetime, timezone, timedelta
import time

In [ ]:
df1 = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\webcrawler\HeatSinkOutput.csv")
df2 = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\webcrawler\MagStabOutput.csv")
df3 = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\webcrawler\GyroStabOutput.csv")

df_merge = pd.concat([df1, df2], ignore_index=True)
df_merge = pd.concat([df_merge, df3], ignore_index=True)
df_merge

In [ ]:
url = "https://esi.evetech.net/latest/universe/regions/"

response = requests.get(url)
region_ids = response.json()
print(f"Regions to scan: {len(region_ids)}")

In [ ]:
contracts = []
for region_id in region_ids:
    total_pages = 1
    current_page = 1
    base_url = "https://esi.evetech.net/latest/contracts/public/{}/".format(region_id)
    params = {"datasource": "tranquility", "page": current_page}
    
    while params["page"] <= total_pages:
        print(base_url)
        print(params)
        response = requests.get(base_url, params=params)
        print(response.headers)
        print(response)
        if total_pages == 1:
            total_pages = int(response.headers.get("X-Pages"))
            
        if response.status_code == 200:
            raw_contracts = response.json()
            item_exchange_contracts = [contract for contract in raw_contracts if contract["type"] == "item_exchange"]
            contracts.extend(item_exchange_contracts)
            params["page"] += 1
        else:
            raise ValueError("Unexpected API Response")
        if "ETag" not in params:
            params["ETag"] = response.headers.get("ETag").strip('\"')    
        

pd.json_normalize(contracts).to_parquet("./api_database/contracts.parquet")

In [ ]:
df_contracts = pd.read_parquet("./api_database/contracts.parquet")[["contract_id", "date_issued"]]

known_abyssal_contracts = pd.read_parquet("./api_database/item_stats.parquet")
known_abyssal_contracts_agg = known_abyssal_contracts.groupby("contract_id", as_index=False).agg({
    "item_id": "nunique"
}).rename(columns={"item_id":"abyssal_count"})

df_contracts = df_contracts.merge(known_abyssal_contracts_agg,how="left", on ="contract_id")
df_contracts["date_issued"] = pd.to_datetime(df_contracts["date_issued"])
df_contracts = df_contracts[(df_contracts["date_issued"] > (pd.to_datetime(datetime.utcnow() - timedelta(days=1)).tz_localize("UTC"))) | (df_contracts["abyssal_count"] > 0)][["contract_id"]]

last_run_items = pd.read_parquet("./api_database/contract_items.parquet")
last_run_contracts = last_run_items.groupby("contract_id", as_index=False).agg({
    "updated_at": "max"
})

contracts_to_scan = df_contracts.merge(last_run_contracts, how="left", on="contract_id")
contracts_to_scan = contracts_to_scan[(contracts_to_scan["updated_at"] < (pd.to_datetime("today") + pd.DateOffset(hours=-12))) | (contracts_to_scan["updated_at"].isna())] ["contract_id"].drop_duplicates(keep="first").tolist()

last_run_items = last_run_items.merge(df_contracts[["contract_id"]].drop_duplicates(keep="first"), how="inner", on="contract_id")
contracts_scanned = last_run_items[last_run_items["updated_at"] > (pd.to_datetime("today") + pd.DateOffset(hours=-12))]

In [ ]:
len(contracts_to_scan)

In [ ]:
items = []
contract_count = 0
print (len(contracts_to_scan))
for contract_id in contracts_to_scan:
    total_pages = 1
    page = 1
    while page <= total_pages:
        contract_count += 1
        base_url = "https://esi.evetech.net/latest/contracts/public/items/{}/".format(contract_id)
        params = {"datasource": "tranquility", "page": page}
                
        print(base_url)
        print(params)
        print(contract_count)
        response = requests.get(base_url, params=params)
        print(response.headers)
        print(response)
        if (total_pages == 1) & (response.headers.get("X-Pages") != None):
            total_pages = int(response.headers.get("X-Pages"))
            print(total_pages)
        
        if response.status_code == 200:
            if response.text != "":
                raw_contract_items = response.json()
                contract_items = [contract_item for contract_item in raw_contract_items]
                print(f"Items found: {len(contract_items)}")
                for contract_item in contract_items:
                    contract_item["contract_id"] = contract_id
                    contract_item["updated_at"] = datetime.utcnow()
                items.extend(contract_items)
        else:
            if int(response.headers.get("X-Esi-Error-Limit-Remain", 100)) < 20:
                break
            elif int(response.headers.get("X-Esi-Error-Limit-Remain", 50)) < 50:
                time.sleep(60)
        page += 1

contracts_scanned = pd.concat([contracts_scanned, pd.json_normalize(items)],ignore_index=True)
contracts_scanned.to_parquet("./api_database/contract_items.parquet")


In [ ]:
known_abyssal_contracts = pd.read_parquet("./api_database/item_stats.parquet")
known_abyssal_contracts_agg = known_abyssal_contracts.groupby(["contract_id"], as_index=False).agg({
    "item_id": "nunique"
}).rename(columns={"item_id":"item_id_count"})

# Clean up affteragg data TODO
contract_items = pd.read_parquet("./api_database/contract_items.parquet")
contract_items = contract_items.merge(known_abyssal_contracts_agg, how="left", on=["contract_id"])
contract_items_to_scan = contract_items[(contract_items["item_id_count"].isna()) & (contract_items["item_id"].notna())]
contract_items_to_scan["item_id"] = contract_items_to_scan["item_id"].astype("int64")
contract_items_to_scan["item_idtype_id"] = contract_items_to_scan["type_id"].astype("int64")

contract_items_scanned = contract_items[(contract_items["item_id_count"].notna())]
# contract_items_scanned = known_abyssal_contracts.merge(contract_items_scanned[["contract_id"]].drop_duplicates(keep="first"), how="inner", on="contract_id")
contract_items_scanned

In [ ]:
item_stats = []

for index, row in contract_items_to_scan.iterrows():
    if row["type_id"] in [49726, 49722, 49730]:
        base_url = f"https://esi.evetech.net/latest/dogma/dynamic/items/{row['type_id']}/{row['item_id']}/"
        params = {"datasource": "tranquility"}
        print(base_url)
        print(params)
        response = requests.get(base_url, params=params)
        print(response.headers)
        print(response)

        if response.status_code == 200:
            row_info = response.json()
            dogma_details = []
            dogma_details = [mod for mod in row_info["dogma_attributes"] if mod["attribute_id"] in [50,64,204]]
            for mod in dogma_details:
                mod["contract_id"] = row["contract_id"]
                mod["type_id"] = row["type_id"]
                mod["item_id"] = row["item_id"]
            item_stats.extend(dogma_details)
        else:
            if int(response.headers["X-Esi-Error-Limit-Remain"]) < 20:
                break
            elif int(response.headers["X-Esi-Error-Limit-Remain"]) < 50:
                time.sleep(60)
                
df_pivoted = pd.json_normalize(item_stats).pivot(index=['contract_id', 'type_id', 'item_id'], columns='attribute_id', values='value').reset_index()
df_pivoted = df_pivoted.rename_axis(None, axis=1)
df_pivoted = df_pivoted.rename(columns={50: "CPU", 64: "Damage" , 204:"ROF"})
output = pd.concat([contract_items_scanned, df_pivoted], ignore_index=True).drop_duplicates(keep="first")
 		
output.to_parquet("./api_database/item_stats.parquet")
output
